<a href="https://colab.research.google.com/github/ccosmin97/idc-prostate-mri-analysis/blob/main/google_cloud_notebooks/MONAI_predict_eval_nnunet_prostate_zonal_regions_prostatex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prostate segmentation on IDC collection -- QIN collection
*   Dataset : [QIN Prostate Repeatability]()
*   Goal : Prostate segmentation using Task24 Prostate nnU-net pre-trained model, T2 input single-modality

In [ ]:
# !sudo apt-get install g++ make git cmake-curses-gui \
#   libblas-dev liblapack-dev libsqlite3-dev \
#   libdcmtk-dev libdlib-dev libfftw3-dev \
#  libinsighttoolkit4-dev \
#   libpng-dev libtiff-dev uuid-dev zlib1g-dev

# Download platimatch binaries

In [ ]:
!wget -O plastimatch.zip https://www.dropbox.com/scl/fi/c2nsgewjw1ppp5nc75oiv/plastimatch.zip?rlkey=ya25vx7bojz7xobqbsr74ujxy&dl=0

--2024-06-10 02:17:28--  https://www.dropbox.com/scl/fi/c2nsgewjw1ppp5nc75oiv/plastimatch.zip?rlkey=ya25vx7bojz7xobqbsr74ujxy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf7d885d715bbc47c67fd8ed2cd.dl.dropboxusercontent.com/cd/0/inline/CUgizNjUE91lYyFYaLLZ2JWv6AKuCtX08iPP6J5fhI3tbGMss98t0tY6R1-wDcjqY5weC8TCLcimDJGwGYeY8nGjGYqWIsygJzcNFmzVRoM618agr1X-lkiWDK5fkGVfXFCGAi7xg03_y0z7zeg1GzDv/file# [following]
--2024-06-10 02:17:29--  https://ucf7d885d715bbc47c67fd8ed2cd.dl.dropboxusercontent.com/cd/0/inline/CUgizNjUE91lYyFYaLLZ2JWv6AKuCtX08iPP6J5fhI3tbGMss98t0tY6R1-wDcjqY5weC8TCLcimDJGwGYeY8nGjGYqWIsygJzcNFmzVRoM618agr1X-lkiWDK5fkGVfXFCGAi7xg03_y0z7zeg1GzDv/file
Resolving ucf7d885d715bbc47c67fd8ed2cd.dl.dropboxusercontent.com (ucf7d885d715bbc47c67fd8ed2cd.dl.dropboxusercontent.com)... 162.125.1.15, 262

In [ ]:
!unzip plastimatch.zip

Archive:  plastimatch.zip
   creating: plastimatch_build/
  inflating: plastimatch_build/libplmregister.a  
  inflating: plastimatch_build/dicom_info  
  inflating: plastimatch_build/libplmsys.a  
  inflating: plastimatch_build/hnd_to_pfm  
  inflating: plastimatch_build/libinih.a  
  inflating: plastimatch_build/libspecfun.a  
  inflating: plastimatch_build/cmake_uninstall.cmake  
  inflating: plastimatch_build/CPackConfig.cmake  
  inflating: plastimatch_build/libplmreconstruct.a  
  inflating: plastimatch_build/cmake_install.cmake  
  inflating: plastimatch_build/plm_config.h  
  inflating: plastimatch_build/libplmclp.a  
  inflating: plastimatch_build/libnkidecompress.a  
  inflating: plastimatch_build/plastimatch  
  inflating: plastimatch_build/xvi_archive  
  inflating: plastimatch_build/dlib_train  
  inflating: plastimatch_build/Makefile  
  inflating: plastimatch_build/run_lcov.sh  
  inflating: plastimatch_build/CPackSourceConfig.cmake  
  inflating: plastimatch_build/liblbf

Check what system dependencies are missing:

In [ ]:
!ldd plastimatch_build/plastimatch

	linux-vdso.so.1 (0x00007ffeee156000)
	libITKIOLSM-4.13.so.1 => not found
	libITKIOBruker-4.13.so.1 => not found
	libITKIOHDF5-4.13.so.1 => not found
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f5aef9aa000)
	libITKIOMINC-4.13.so.1 => not found
	libITKIOMRC-4.13.so.1 => not found
	libdcmjpeg.so.16 => not found
	libdlib.so.19 => not found
	libfftw3.so.3 => not found
	libdcmimgle.so.16 => not found
	libdcmdata.so.16 => not found
	liboflog.so.16 => not found
	libofstd.so.16 => not found
	libITKIOBMP-4.13.so.1 => not found
	libITKIOGDCM-4.13.so.1 => not found
	libITKIOGIPL-4.13.so.1 => not found
	libITKIOJPEG-4.13.so.1 => not found
	libITKIOMeta-4.13.so.1 => not found
	libITKIONIFTI-4.13.so.1 => not found
	libITKIONRRD-4.13.so.1 => not found
	libITKIOPNG-4.13.so.1 => not found
	libITKIOTIFF-4.13.so.1 => not found
	libITKIOVTK-4.13.so.1 => not found
	libITKIOBioRad-4.13.so.1 => not found
	libITKIOStimulate-4.13.so.1 => not found
	libITKOptimizers-4.13.so.1 => not found
	libITKIOGE-

Install the packages that solve these dependencies:

In [ ]:
!sudo apt update && sudo apt install libinsighttoolkit4-dev libdcmtk16 libdlib19 libfftw3-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http:

In [ ]:
!pip install itk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 79.0 MB/s eta 0:00:00


---

In [ ]:
!mv plastimatch_build/plastimatch ./plastimatch
!rm -r plastimatch_build
!mv

mv: missing file operand
Try 'mv --help' for more information.


In [ ]:
!./plastimatch resample

Error.  Please specify an input file using the --input option
Usage: plastimatch resample [options]
Options:
      --default-value <arg>   value to set for pixels with unknown value, 
                               default is 0 
      --dim <arg>             size of output image in voxels "x [y z]" 
      --direction-cosines <arg>   
                              oriention of x, y, and z axes; Specify either 
                               preset value, 
                               {identity,rotated-{1,2,3},sheared}, or 9 digit 
                               matrix string "a b c d e f g h i" 
  -F, --fixed <arg>           fixed image (match output size to this image) 
  -h, --help                  display this help message 
      --input <arg>           input directory or filename; can be an image or 
                               vector field 
      --interpolation <arg>   interpolation type, either "nn" or "linear", 
                               default is linear 
      --orig

In [ ]:
!cp ./plastimatch /usr/local/bin/plastimatch

# Global variables

## For SR and radiomics computation

In [ ]:
#nnunet code mapping
import pandas as pd
#radiomics df
!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
nnunet_shape_features_code_mapping_df = pd.read_csv("nnunet_shape_features_code_mapping.csv")

--2024-06-10 02:18:34--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1053 (1.0K) [text/plain]
Saving to: ‘nnunet_shape_features_code_mapping.csv’

nnunet_shape_featur 100%[===================>]   1.03K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-06-10 02:18:34 (55.5 MB/s) - ‘nnunet_shape_features_code_mapping.csv’ saved [1053/1053]



In [ ]:
nnunet_segments_code_mapping_df = pd.DataFrame.from_dict({'segment': ['Prostate'],
                                                          'Finding_CodingSchemeDesignator': ['SCT'], 'Finding_CodeValue' : ['123037004'], 'Finding_CodeMeaning' : ['Anatomical Structure'],
                                                          'FindingSite_CodingSchemeDesignator' : ['SCT'], 'FindingSite_CodeValue' : ['41216001'], 'FindingSite_CodeMeaning' : ['Prostate']})
nnunet_segments_code_mapping_df.head()

,segment,Finding_CodingSchemeDesignator,Finding_CodeValue,Finding_CodeMeaning,FindingSite_CodingSchemeDesignator,FindingSite_CodeValue,FindingSite_CodeMeaning
0,Prostate,SCT,123037004,Anatomical Structure,SCT,41216001,Prostate


In [ ]:
idc_seg_segments_code_mapping_df = pd.DataFrame.from_dict({'segment': ['Prostate', 'Peripheral zone of prostate', 'Transition zone of prostate'],
                                                          'Finding_CodingSchemeDesignator': ['SCT', 'SCT', 'SCT'], 'Finding_CodeValue' : ['123037004', '123037004', '123037004'],
                                                           'Finding_CodeMeaning' : ['Anatomical Structure', 'Anatomical Structure', 'Anatomical Structure'],
                                                          'FindingSite_CodingSchemeDesignator' : ['SCT', 'SCT', 'SCT'],
                                                           'FindingSite_CodeValue' : ['41216001', '279706003', '399384005'],
                                                           'FindingSite_CodeMeaning' : ['Prostate', 'Peripheral zone of prostate', 'Transition zone of prostate']})
idc_seg_segments_code_mapping_df.head()

,segment,Finding_CodingSchemeDesignator,Finding_CodeValue,Finding_CodeMeaning,FindingSite_CodingSchemeDesignator,FindingSite_CodeValue,FindingSite_CodeMeaning
0,Prostate,SCT,123037004,Anatomical Structure,SCT,41216001,Prostate
1,Peripheral zone of prostate,SCT,123037004,Anatomical Structure,SCT,279706003,Peripheral zone of prostate
2,Transition zone of prostate,SCT,123037004,Anatomical Structure,SCT,399384005,Transition zone of prostate


## Variables used for resampling -- inference -- setup labelID for ground truth segs

In [ ]:
# IDC collection for paths setup
collection_name = "prostatex" #up to user
##nnunet
model_type = '3d_fullres'#other options are '2d',..
nnunet_labelID = 1 #unique values for nifti segmentation volumes predicted, here 1 is prostate and 0 is background
desired_grid = "T2" #Other option is "ADC"

## Global paths

Here we create the necessary folders, IDC folders for download MR T2 data, and nnunet workspace.

In [ ]:
import os
#qin_prostate_repeatibility
os.environ["prostatex_root"] = os.path.join(os.getcwd(), "prostateX")
os.environ["prostatex_dicom"] = os.path.join(os.environ["prostatex_root"], "dicom")
os.environ["prostatex_nii"] = os.path.join(os.environ["prostatex_root"], "nii")
os.environ["prostatex_nii_formatted"] = os.path.join(os.environ["prostatex_root"], "nii_formatted")
os.environ["prostatex_nrrd"] = os.path.join(os.environ["prostatex_root"], "nrrd")
#SEG IDC
#seg files
# os.environ["stl_labelmap_root"] = os.path.join(os.getcwd(), "stl_labelmap")
# os.environ["stl_labelmap_dicom_seg"] = os.path.join(os.environ["stl_labelmap_root"], "dicom_seg")
# os.environ["stl_labelmap_nii_seg"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg")
# os.environ["stl_labelmap_nii_seg_sr_json"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_sr_json")
# os.environ["stl_labelmap_nii_seg_sr"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_sr")
# os.environ["stl_labelmap_nii_seg_formatted"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_formatted")
# os.environ["stl_labelmap_nii_seg_resampled"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_resampled")
# os.environ["stl_labelmap_nii_seg_features"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_features")
# os.environ["stl_labelmap_nii_seg_split_temp_resampled"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_split_temp_resampled")
#
os.environ["prostatex_dicom_seg"] = os.path.join(os.environ["prostatex_root"], "dicom_seg")
os.environ["prostatex_nii_seg"] = os.path.join(os.environ["prostatex_root"], "nii_seg")
os.environ["prostatex_nii_seg_sr_json"] = os.path.join(os.environ["prostatex_root"], "nii_seg_sr_json")
os.environ["prostatex_nii_seg_sr"] = os.path.join(os.environ["prostatex_root"], "nii_seg_sr")
os.environ["prostatex_nii_seg_formatted"] = os.path.join(os.environ["prostatex_root"], "nii_seg_formatted")
os.environ["prostatex_nii_seg_resampled"] = os.path.join(os.environ["prostatex_root"], "nii_seg_resampled")
os.environ["prostatex_nii_seg_features"] = os.path.join(os.environ["prostatex_root"], "nii_seg_features")
os.environ["prostatex_nii_seg_split_temp_resampled"] = os.path.join(os.environ["prostatex_root"], "nii_seg_split_temp_resampled")
#monai
os.environ["monai_workspace"] = "monai_workspace"
os.environ["monai_workspace_eval"] = os.path.join(os.environ["monai_workspace"], "eval")
os.environ["monai_workspace_eval_resampled"] = os.path.join(os.environ["monai_workspace"], "eval_resampled/")
os.environ["monai_workspace_eval_resampled_split"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_split/")
os.environ["monai_workspace_eval_resampled_split_labelID"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_split_labelID")
os.environ["monai_workspace_eval_resampled_dcm"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_dcm/")
os.environ["monai_workspace_eval_resampled_dcm_to_nifti"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_dcm_to_nifti/")
os.environ["monai_workspace_eval_features"] = os.path.join(os.environ["monai_workspace"], "eval_features")
os.environ["monai_workspace_eval_split_temp"] = os.path.join(os.environ["monai_workspace"], "temp_split")
os.environ["monai_workspace_eval_sr"] = os.path.join(os.environ["monai_workspace"], "structured_reports")
os.environ["monai_workspace_eval_sr_json"] = os.path.join(os.environ["monai_workspace"], "structured_reports_json")

os.environ["monai_workspace_dataset"] = os.path.join(os.environ["monai_workspace"], "dataset/")
os.environ["monai_workspace_model_dir"] = os.path.join(os.environ["monai_workspace"], "models")
os.environ["monai_workspace_model_path"] = os.path.join(os.environ["monai_workspace"], "models", "anatomy.pt")
#path where model pre-trained weights are stored
# os.environ["PATH_TO_MODEL_FILE"] = os.path.join(os.environ["nnUNet"], "models", "Task024_Prostate.zip")
# #misc
# os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_DL")
# os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_DL", "Sorted")
# os.environ["logs"] = os.path.join(os.getcwd(), "logs")
###UPDATED
#idc data
os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_Downloads")
os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_Downloads_Sorted")
os.environ["IDC_IMG_Downloads"] = os.path.join(os.getcwd(), "IDC_IMG_Downloads")
os.environ["IDC_IMG_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_IMG_Downloads_Sorted")
#evaluation
os.environ["mri_us_biopsy_analysis"] = os.path.join(os.getcwd(), "mri_us_biopsy_analysis")
os.environ["mri_us_biopsy_analysis_results"] = os.path.join(os.environ["mri_us_biopsy_analysis"], "results")
os.environ["mri_us_biopsy_analysis_verbose"] = os.path.join(os.environ["mri_us_biopsy_analysis"], "results_verbose")
os.environ["logs"] = "logs"
#create dirs for specific folders names
for key, path in os.environ.items():
  check_patterns = [True for el in ["prostatex", "nnunet", "IDC", "nnUNet", "stl_labelmap", "monai_workspace_model_dir",
                                    "logs", "RESULTS_FOLDER", "monai_workspace_eval", "monai_workspace_dataset",
                                    "stl_labelmaps", "mri_us_biopsy_analysis"] if el in key]
  if True in check_patterns:
    !mkdir -p $path

# Auth login

##Colab

Google Colab authentification, necessary step to link account to GCP project to retrieve IDC data through BigQuery

In [ ]:
#colab
from google.colab import auth
auth.authenticate_user()

# Setup GCP Project ID

In [ ]:
project_id = "idc-sandbox-003"
os.environ["GCP_PROJECT_ID"] = project_id

# Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies.

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

## GPU checks

In [ ]:
# check wether the Colab Instance was correctly initialized with a GPU instance
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

if not has_gpu:
  print("Your Colab VM does not have a GPU - check \"Runtime > Change runtime type\"")

In [ ]:
# check which model of GPU the notebook is equipped with - a Tesla K80 or T4
# T4 is the best performing on the two - and can about half the GPU processing time

!nvidia-smi

Mon Jun 10 02:18:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install command-line tools


[Plastimatch](https://plastimatch.org/index.html) is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one).

In [ ]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [ ]:
!echo $(plastimatch --version)

plastimatch version 1.9.4-46-g950dde17


[dcmqi](https://github.com/QIICR/dcmqi) is an open source library that can help with the conversion between imaging research formats and the standard DICOM representation for image analysis results. More specifically, you can use dcmqi convert DICOM Segmentation objects (DICOM SEG) into research formats, such as NIfTI and NRRD.

In [ ]:
%%capture
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz
!tar zxvf dcmqi-1.2.5-linux.tar.gz
!cp dcmqi-1.2.5-linux/bin/* /usr/local/bin/

Finally, we are going to install [Subversion](https://subversion.apache.org/), a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [ ]:
%%capture

!sudo apt install subversion

In [ ]:
!echo $(svn --version | head -n 2)

svn, version 1.14.1 (r1886195) compiled May 21 2022, 10:52:35 on x86_64-pc-linux-gnu


Download monai bundle from github

In [ ]:
!git clone https://github.com/Project-MONAI/model-zoo/

Cloning into 'model-zoo'...
remote: Enumerating objects: 4437, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 4437 (delta 20), reused 30 (delta 12), pack-reused 4380
Receiving objects: 100% (4437/4437), 36.04 MiB | 32.66 MiB/s, done.
Resolving deltas: 100% (3004/3004), done.


## Install Python packages

In [ ]:
!git clone https://github.com/herrmannlab/highdicom.git
#!cd highdicom && python setup.py install
!cd highdicom && pip install .

Cloning into 'highdicom'...
remote: Enumerating objects: 7636, done.
remote: Counting objects: 100% (1225/1225), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 7636 (delta 997), reused 1125 (delta 955), pack-reused 6411
Receiving objects: 100% (7636/7636), 3.84 MiB | 7.66 MiB/s, done.
Resolving deltas: 100% (5052/5052), done.
Processing /content/highdicom
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 13.6 MB/s eta 0:00:00
  Created wheel for highdicom: filename=highdicom-0.22.0-py3-none-any.whl size=825743 sha256=e5b03b2d04652b5672af41a1af3bb98e911429daa37fd41ff0ca049fe007798c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gznl101q/wheels/85/80/88/55471f8e67c8ff2f09dd805b1c472392006bfa74851bec2b6d
Successfully built highdicom


In [ ]:
%%capture
!pip install nnunet
!pip install pydicom
!pip install seg-metrics
!pip install nibabel
!pip install dcm2niix
!pip install SimpleITK
!pip install pyradiomics
!pip install pandas
!pip install pyplastimatch

In [ ]:
!pip install -U "monai[all]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━

Next we set up few things to help with visualization of the segmentations later.

In [ ]:
import os
import sys
import shutil
import csv
import random

import os
import glob
import csv
import json

import nibabel as nib

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk
from SimpleITK import GetArrayViewFromImage as ArrayView
from functools import partial

import seg_metrics.seg_metrics as sg
# from medpy.metric.binary import dc as dice_coef
# from medpy.metric.binary import hd as hausdorff_distance
# from medpy.metric.binary import asd as asd
# from medpy.metric.binary import precision
# from medpy.metric.binary import recall
# from medpy.metric.binary import specificity
# from medpy.metric.binary import sensitivity
# from medpy.metric.binary import true_positive_rate

import radiomics
from radiomics import featureextractor

# from medpy.filter.binary import largest_connected_component

# use the "tensorflow_version" magic to make sure TF 1.x is imported
# %tensorflow_version 2.x
# import tensorflow as tf
# import keras

from monai.bundle import ConfigParser, run

In [ ]:
# Packages for the structured report

import highdicom

from pathlib import Path

import highdicom as hd

from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion,
    SourceImageForMeasurement,
    SourceImageForMeasurementGroup
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    MeasurementsAndQualitativeEvaluations,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation,
    ImageLibrary,
    ImageLibraryEntryDescriptors
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [ ]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch pyplastimatch

svn: E170013: Unable to connect to a repository at URL 'https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch'
svn: E160013: '/AIM-Harvard/pyplastimatch/trunk/pyplastimatch' path not found


In [ ]:
# dicomsort is the pythong package that can sort DICOM files into
# folder organization based on user-specified DICOM attributes
!git clone https://github.com/pieper/dicomsort.git

Cloning into 'dicomsort'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 169 (delta 23), reused 33 (delta 16), pack-reused 126
Receiving objects: 100% (169/169), 87.84 KiB | 3.14 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.utils import data as data_utils

# Custom functions

## Pre-processing / post-processing

In [ ]:
def resample_idc_seg_nii(input_folder_seg_nii, output_folder_seg_nii, search_path_ref_nii_file):
  for seg_nii in glob.glob(os.path.join(input_folder_seg_nii, "*.nii.gz")):
    print(f"{desired_grid}_path found : {search_path_ref_nii_file}")
    print(f"pred path : {seg_nii}")
    output = os.path.join(output_folder_seg_nii, f"{seg_nii.split('/')[-1]}")
    resample_args_to_t2_origin = {"input" : seg_nii,#change to pred path if no largest_component_retrieval necessary
                          "output" : output,
                          "fixed" : search_path_ref_nii_file,
                          "interpolation" : "nn"}
    print(f"Resampling seg named : {seg_nii.split('/')[-1]} with fixed grid : {search_path_ref_nii_file.split('/')[-1]} ...")
    pypla.resample(verbose = False, **resample_args_to_t2_origin)

In [ ]:
def change_labelID_seg_nii(in_seg, labelID, output_folder_path):
  input_labelmap_image = sitk.ReadImage(in_seg)
  input_labelmap_arr = sitk.GetArrayFromImage(input_labelmap_image)
  temp_arr = input_labelmap_arr
  temp_arr[temp_arr != 0] = labelID
  # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
  res_img = sitk.GetImageFromArray(temp_arr)
  res_img.CopyInformation(input_labelmap_image)
  sitk.WriteImage(res_img, os.path.join(output_folder_path,in_seg.split('/')[-1]))

In [ ]:
def combine_prostate_regions(input_path_nii_pred, output_path_folder, filename):
  # Starting with SimpleITK
  img_pred = sitk.ReadImage(input_path_nii_pred)
  npy_pred = sitk.GetArrayFromImage(img_pred)
  # Custom Python code working on a numpy array.
  npy_pred[npy_pred != 0] = 1
  # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
  res_img = sitk.GetImageFromArray(npy_pred)
  res_img.CopyInformation(img_pred)
  # Save output
  sitk.WriteImage(res_img, os.path.join(output_path_folder, filename))

In [ ]:
def download_idc_data_serie_uid_seg(df_in, out_path, idc_downloads_path, idc_downloads_sorted_path):
  # save the list of GCS URLs into a file
  selection_manifest = os.path.join(idc_downloads_path, "idc_manifest.txt")
  df_in["gcs_url"].to_csv(selection_manifest, header=False, index=False)
  # let's make sure the download folder is clean, in case you ran this cell earlier
  # for a different dataset
  # !rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}
  !cat {selection_manifest} | gsutil -m cp -I {idc_downloads_path}
  !python dicomsort/dicomsort.py -k -u {idc_downloads_path} {idc_downloads_sorted_path}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  # !rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}
  in_mv = idc_downloads_sorted_path+'/*'
  !mv $in_mv $out_path
  #{os.environ["qin_prostate_rep_dicom"]}
  # convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
  #                 output_path=os.environ["qin_prostate_rep_root"], idc_df=selection_df)

In [ ]:
def convert_seg_to_nii(input_path, output_path):
  if not os.path.exists(output_path):
    !mkdir -p $output_path

  print(f'input path : {input_path}')
  print(f'output_path : {output_path}')
  !segimage2itkimage --inputDICOM $input_path --outputDirectory $output_path \
  --outputType nii

In [ ]:
def split_seg_nii(input_labelmap, output_folder_path, dic_labelID):
  for labelMeaning, unique_label in dic_labelID.items():
    input_labelmap_image = sitk.ReadImage(input_labelmap)
    input_labelmap_arr = sitk.GetArrayFromImage(input_labelmap_image)
    temp_arr = input_labelmap_arr
    temp_arr[temp_arr != int(unique_label)] = 0
    temp_arr[temp_arr == int(unique_label)] = 1
    # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
    res_img = sitk.GetImageFromArray(temp_arr)
    res_img.CopyInformation(input_labelmap_image)
    sitk.WriteImage(res_img, os.path.join(output_folder_path, f"{labelMeaning}.nii.gz"))

In [ ]:
def resample_idc_seg_nii(input_folder_seg_nii, output_folder_seg_nii, search_path_ref_nii_file):
  for seg_nii in glob.glob(os.path.join(input_folder_seg_nii, "*.nii.gz")):
    print(f"{desired_grid}_path found : {search_path_ref_nii_file}")
    print(f"pred path : {seg_nii}")
    output = os.path.join(output_folder_seg_nii, f"{seg_nii.split('/')[-1]}")
    resample_args_to_t2_origin = {"input" : seg_nii,#change to pred path if no largest_component_retrieval necessary
                          "output" : output,
                          "fixed" : search_path_ref_nii_file,
                          "interpolation" : "nn"}
    print(f"Resampling seg named : {seg_nii.split('/')[-1]} with fixed grid : {search_path_ref_nii_file.split('/')[-1]} ...")
    pypla.resample(verbose = True, **resample_args_to_t2_origin)

In [ ]:
def resample_itk_identity(mask_path : str,
                          image_path : str, out_image_path : str):
  image = sitk.ReadImage(image_path)
  mask = sitk.ReadImage(mask_path)
  result_image = sitk.Resample(mask, image, sitk.Transform(),sitk.sitkNearestNeighbor, 0, mask.GetPixelID())
  sitk.WriteImage(result_image, out_image_path)

In [ ]:
def convert_image_dcm_to_nii(input_path, output_path_root, prefix="", format="nii"):
  """
  Conversion of DICOM MR data to NIFTI using dcm2niix
  input_path : str, folder containing DICOM instances .dcm
  output_path_root : str, output folder
  prefix : str, prefix for output file name
  """
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  !dcm2niix -z y -m y -f %i_{prefix} -o $output_path_root $input_path

In [ ]:
def convert_image_dcm_to_nrrd(input_path, output_path_root, format="nrrd", prefix=""):
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  # !dcm2niix -e y -z y -m y -f %i_{output_name_file_suffix} -o $output_path_root $input_path
  out_path_file = f"{output_path_root}/{prefix}.nrrd"
  !plastimatch convert \
  --input $input_path \
  --output-img $out_path_file

In [ ]:
def convert_dcm_sorted(input_path,output_path, idc_df):
  """
  Convert DCM MR sorted by PatID/StudyUID/SerieUID in NIFTI format
  input_path : str, root folder containing all series sorted by patid/studyUID/serieUID/sopUID.dcm
  output_path : str, root folder of where to store converted MR T2 series
  idc_df : pandas.Dataframe, contains IDC metadata to retrieve modality information etc per serieUID
  """
  for serie_folder in sorted(glob.glob(os.path.join(input_path, "**", "**", "*"))):#grab serie folder in sorted MR IDC DICOM data folder
    path_serie_dcm_lst = glob.glob(os.path.join(serie_folder, "*.dcm"))
    modality = idc_df[idc_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["Modality"].iloc[0]#get modality per serie
    seriesInstanceUID = serie_folder.split("/")[-1]
    studyInstanceUID = serie_folder.split("/")[-2]
    patientID = serie_folder.split("/")[-3]
    print(f"Serie processed : {serie_folder}")
    print(f"SeriesDescription : {pydicom.read_file(glob.glob(os.path.join(serie_folder, '*.dcm'))[0]).SeriesDescription}")
    print(f"Modality : {pydicom.read_file(glob.glob(os.path.join(serie_folder, '*.dcm'))[0]).Modality}")
    #convert to NIFTI
    convert_image_dcm_to_nii(input_path=serie_folder,
                           output_path_root=os.path.join(output_path, "nii"),
                           prefix=f"{seriesInstanceUID}", format="nii")
    convert_image_dcm_to_nrrd(input_path=serie_folder,
                           output_path_root=os.path.join(output_path, "nrrd"),
                           prefix=f"{patientID}_{seriesInstanceUID}", format="nrrd")

In [ ]:
def download_idc_data_serie_uid(idc_t2_df, idc_downloads_path, idc_downloads_sorted_path, idc_dicom_path, idc_nii_path):
  """
  Download IDC data from gcs_urls retrieved from BigQuery
  idc_df : pandas.Dataframe, contains MR series DICOM metadata and gcs_urls
  """
  # save the list of GCS URLs into a file
  selection_manifest = os.path.join(idc_downloads_path, "idc_manifest.txt")
  idc_t2_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)
  out_folder = idc_downloads_path
  !cat {selection_manifest} | gsutil -m cp -Ir $out_folder #download gcs_urls into output_folder
  #sort dicom donwload data per PatID/studyUID/serieUID
  # !python dicomsort/dicomsort.py -k -u {os.environ["IDC_Downloads"]} {os.environ["IDC_Downloads_Sorted"]}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  # !mv {os.environ['IDC_Downloads_Sorted']+'/*'} {os.environ['prostatex_dicom']}#move sorted IDC DICOM data to prostatex_dicom folder
  !python dicomsort/dicomsort.py -k -u $out_folder {idc_downloads_sorted_path}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  in_mv = idc_downloads_sorted_path+'/*'
  out_mv = idc_dicom_path
  !mv $in_mv $out_mv
  #Convert to NIFTI
  convert_dcm_sorted(input_path=idc_dicom_path,
                  output_path=os.environ["prostatex_root"], idc_df=idc_t2_df)
  return idc_t2_df

In [ ]:
def reformat_image_nnunet(patID, serieUID, images_nii_folder, out_path_folder):
  """
  formats input MR T2 data into nnunet ready format, caseID_Modality.nii.gz, where modality here is T2==0000
  """
  #reformats images to correct format,
  for mr_vol in glob.glob(os.path.join(images_nii_folder, f"{patID}_{serieUID}*.nii.gz")):#get MR T2 NIFTI volume
    serieUID = mr_vol.split('/')[-1].split("_")[1].replace(".nii.gz","")#.split(".")[0]
    patientID = mr_vol.split('/')[-1].split("_")[0]
    nnunet_idx = "0000" #if "T2" in mr_vol.split('/')[-2] else "0001"#0000 for T2 and 0001 for ADC
    nnunet_path = os.path.join(out_path_folder,
                                "_".join([patientID, serieUID, nnunet_idx]) + ".nii.gz")
    !cp $mr_vol $nnunet_path

In [ ]:
def resample_preds(input_path_nnunet_preds="", input_path_t2_idc="", output_path="", suffix_ref=".nrrd"):
  """
  Resampling of nnunet_preds to reference MR T2 volumes used as input image
  input_path_nnunet_preds : str, folder containing nnunet preds
  input_path_t2_idc : str, folder containing NIFTI IDC MR T2 volumes
  output_path : str, folder output
  """
  for pred_path in sorted(glob.glob(os.path.join(input_path_nnunet_preds, "**", "*.nii.gz"), recursive=True)):
    search_t2_path = os.path.join(input_path_t2_idc, \
                                  f"{pred_path.split('/')[-1].split('_')[0]}_{pred_path.split('/')[-1].split('_')[1].replace('.nii.gz','')}*{suffix_ref}") #PatientID_SerieUID.nii.gz
                                      #get serieUID
    print(f"search path for {desired_grid} : {search_t2_path}")
    t2_path = glob.glob(search_t2_path, recursive=True)[0]
    print(f"{desired_grid}_path found : {t2_path}")
    print(f"pred path : {pred_path}")
    resample_args_to_t2_origin = {"input" : pred_path,#change to pred path if no largest_component_retrieval necessary
                          "output" : os.path.join(output_path,
                                                  f"{pred_path.split('/')[-1][:-7]}.nii.gz"),
                          "fixed" : t2_path,
                          "interpolation" : "nn"}

    pypla.resample(verbose = False, **resample_args_to_t2_origin)
    print()

In [ ]:
def resample_seg(input_path_labelmap, input_path_nnunet_preds="", input_path_t2_idc="", output_path=""):
  """
  Resampling of nnunet_preds to reference MR T2 volumes used as input image
  input_path_nnunet_preds : str, folder containing nnunet preds
  input_path_t2_idc : str, folder containing NIFTI IDC MR T2 volumes
  output_path : str, folder output
  """
  pred_path = glob.glob(os.path.join(input_path_nnunet_preds, "*.nii.gz"))[0]
  for labelmap_file in sorted(glob.glob(os.path.join(input_path_labelmap, "*.nii.gz"))):
    search_t2_path = os.path.join(input_path_t2_idc, \
                                  f"{pred_path.split('/')[-1].split('_')[0]}_{pred_path.split('/')[-1].split('_')[1].replace('.nii.gz','')}*.nii.gz") #PatientID_SerieUID.nii.gz
                                      #get serieUID
    print(f"search path for {desired_grid} : {search_t2_path}")
    t2_path = glob.glob(search_t2_path, recursive=True)[0]
    print(f"{desired_grid}_path found : {t2_path}")
    print(f"pred path : {pred_path}")
    resample_args_to_t2_origin = {"input" : labelmap_file,#change to pred path if no largest_component_retrieval necessary
                          "output" : os.path.join(output_path,
                                                  f"{pred_path.split('/')[-1][:-7]}_resampled_seg.nii.gz"),
                          "fixed" : t2_path,
                          "interpolation" : "nn"}

    path_log = os.path.join(os.environ["logs"], 'log_pypla_res_pred' + pred_path.split('/')[-1].split('.')[0] + '.txt')
    !touch $path_log
    pypla.resample(verbose = False, **resample_args_to_t2_origin, path_to_log_file=path_log)
    print()

In [ ]:
def resample_seg_idc(input_path_labelmap, input_path_nnunet_preds="", input_path_t2_idc="", output_path=""):
  pass
  #resample each label seg nii with original mr


In [ ]:
def seg_nii_to_dicom(patID, studyUID, refSerieUID, json_metadata_dcm_path = "", input_path_nii="", input_path_dcm_idc="", output_path_root=""):
  """
  Convert of NIFTI binary segmentations of nnunet to SEG DICOM object
  idc_df : pandas.Dataframe, resulting from Bigquery
  input_path_nii : str, nnunet_preds folder
  input_path_dcm_idc : str,  folder containing DCM IDC MR T2 objects
  output_path_root : str, output_path root folder
  """
  assert os.path.exists(input_path_nii)
  assert os.path.exists(input_path_dcm_idc)
  !mkdir -p $output_path_root
  for nii_seg_pred in glob.glob(os.path.join(input_path_nii, '*.nii.gz')):
    patID = nii_seg_pred.split('/')[-1].split('_')[0]
    study_mr_t2_serieUID = nii_seg_pred.split('/')[-1].split('_')[1].replace(".nii.gz","")
    #find t2 dcm folder
    t2_dcm_folder = glob.glob(os.path.join(input_path_dcm_idc, patID, "**", study_mr_t2_serieUID))[0]
    #find seg dcm file
    # find nii seg folder == preds resampled
    assert os.path.exists(t2_dcm_folder)
    print('\nConverting...')
    print(f'pred nnunet processed : {nii_seg_pred}')
    print(f't2_dcm_folder : {t2_dcm_folder}')
    output_path = os.path.join(output_path_root, '_'.join([patID, study_mr_t2_serieUID])+'.dcm')
    #convert nii pred to dcm
    !itkimage2segimage --inputImageList $nii_seg_pred \
    --inputDICOMDirectory  $t2_dcm_folder \
    --inputMetadata  $json_metadata_dcm_path \
    --outputDICOM $output_path
    print("Done!")

## Evaluation functions

In [ ]:
def compute_dice(pred_path, gt_path):
 #computation of dice score
  out_plast = !plastimatch dice --dice "{gt_path}" "{pred_path}"
  dice_score = [el[1] for el in out_plast.fields() if "DICE" in el[0]][0]#formatting by plastimatch dice output, retrieve of DSC
  return float(dice_score)

In [ ]:
def compute_hsdff_95(pred_path, gt_path, labelID=1):
  out_plast = !plastimatch dice --hausdorff "{gt_path}" "{pred_path}"
  hsdff = [el[5] for el in out_plast.fields() if "Percent" in el[0] and "distance" in el[3]][0]#formatting by plastimatch hsdff output, retrieve of hsdff
  return float(hsdff)

In [ ]:
def compute_hsdff_regular(pred_path, gt_path):
  #computation of dice score
  out_plast = !plastimatch dice --hausdorff "{gt_path}" "{pred_path}"
  hsdff = [el[3] for el in out_plast.fields() if "Hausdorff" in el[0] and "distance" in el[1]][0]#formatting by plastimatch hsdff output, retrieve of hsdff
  return float(hsdff)

In [ ]:
#https://github.com/Jingnan-Jia/segmentation_metrics
def compute_avg_surface_dist(pred_path, gt_path):
  # for only one metrics
  image_gt = sitk.ReadImage(gt_path)
  image_pred = sitk.ReadImage(pred_path)
  gt_spacing = image_gt.GetSpacing()
  gt_size = image_gt.GetSize()
  new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(gt_size, gt_spacing, image_pred.GetSpacing())]
  gt_image_new  = sitk.Resample(image_gt, new_size, sitk.Transform(), sitk.sitkNearestNeighbor,
                        image_gt.GetOrigin(), image_pred.GetSpacing(), image_gt.GetDirection(), 0,
                        image_gt.GetPixelID())
  metrics = sg.write_metrics(labels=[1],  # exclude background if needed -- here only 1 label, prostate
                    gdth_img=gt_image_new,
                    pred_img=sitk.ReadImage(pred_path),
                    csv_file=None,
                    spacing=list(sitk.ReadImage(pred_path).GetSpacing()),
                    metrics='msd')
  return metrics

In [ ]:
def compute_true_positive(pred_path, gt_path):
  out_plast = !plastimatch dice --all "{gt_path}" "{pred_path}"
  return [el[1] for el in out_plast.fields() if "TP" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [ ]:
def compute_true_negative(pred_path, gt_path):
  out_plast = !plastimatch dice --all "{gt_path}" "{pred_path}"
  return [el[1] for el in out_plast.fields() if "TN" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [ ]:
def compute_false_negative(pred_path, gt_path):
  out_plast = !plastimatch dice --all "{gt_path}" "{pred_path}"
  return [el[1] for el in out_plast.fields() if "FN" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [ ]:
def compute_false_positive(pred_path, gt_path):
  out_plast = !plastimatch dice --all "{gt_path}" "{pred_path}"
  return [el[1] for el in out_plast.fields() if "FP" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [ ]:
def compute_sensitivity(pred_path, gt_path):
  out_plast = !plastimatch dice --all "{gt_path}" "{pred_path}"
  return [el[1] for el in out_plast.fields() if "SE" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [ ]:
def compute_specificity(pred_path, gt_path):
  out_plast = !plastimatch dice --all "{gt_path}" "{pred_path}"
  return [el[1] for el in out_plast.fields() if "SP" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

## nnUNet 3D shape features SR creation

In [ ]:
def get_label_and_names_from_metadata_json(dicomseg_json):

  """Returns two lists containing the label values and the corresponding
     CodeMeaning values

  Inputs:
    dicomseg_json : metajson file

  Outputs:
    label_values  : label values from the metajson file
    label_names   : the corresponding CodeMeaning values

  """

  f = open(dicomseg_json)
  meta_json = json.load(f)

  print(meta_json)

  num_regions = len(meta_json['segmentAttributes'][0])
  print ('num_regions: ' + str(num_regions))

  label_values = []
  label_names = []
  for n in range(0,num_regions):
    # label_values.append(n)
    label_value = meta_json['segmentAttributes'][0][n]['labelID']
    label_name = meta_json['segmentAttributes'][0][n]['SegmentedPropertyTypeCodeSequence']['CodeMeaning']
    label_values.append(label_value)
    label_names.append(label_name)

  return label_values, label_names

In [ ]:
def split_nii(input_file, output_directory, label_names):

  """Function to split a single multilabel nii into individual nii files. Used
     for pyradiomics feature extraction.

  Inputs:
    input_file       : input multi-label nii file
    output_directory : where to save the individual nii segments
    label_names      : the names of the labels that correspond to the order of
                       the values in the nii input_file

  Outputs:
    saves the individual nii files to the output_directory

  """

  if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

  # save with the values in the files
  nii = nib.load(input_file)
  header = nii.header
  img = nii.get_fdata()
  unique_labels = list(np.unique(img))
  unique_labels.remove(0) # remove the background

  # split and save
  num_labels = len(unique_labels)
  for n in range(0,num_labels):
    ind = np.where(img==unique_labels[n])
    vol = np.zeros((img.shape))
    vol[ind] = 1
    new_img = nib.Nifti1Image(vol, nii.affine, nii.header)
    output_filename = os.path.join(output_directory, label_names[n] + '.nii.gz')
    nib.save(new_img, output_filename)

  return

In [ ]:
def compute_pyradiomics_3D_features(ct_nifti_path,
                                    label_values,
                                    label_names,
                                    split_pred_nifti_path,
                                    nnunet_shape_features_code_mapping_df):

  """Function to compute pyradiomics 3D features for each label in a nifti file.


  Inputs:
    ct_nifti_path            : the CT nifti file
    label_values             : the label value for each of the segments from the json file
    label_names              : the corresponding label name for each of the segments
    split_pred_nifti_path    : where to save the individual nii segments needed
                               for pyradiomics
    nnunet_shape_features_code_mapping_df : the df where we will obtain the
                                            list of the shape features to
                                            compute

  Outputs:
    Writes the features_csv_path_nnunet to disk.

  """

  # Get the names of the features from the nnunet_shape_features_code_mapping_df
  shape_features = list(nnunet_shape_features_code_mapping_df['shape_feature'].values)

  # Instantiate the extractor and modify the settings to keep the 3D shape features
  extractor = featureextractor.RadiomicsFeatureExtractor(correctMask=True, geometryTolerance=float(1e-2))
  extractor.settings['minimumROIDimensions'] = 3
  extractor.disableAllFeatures()
  extractor.enableFeaturesByName(shape=shape_features)

  # Calculate features for each label and create a dataframe
  num_labels = len(label_names)#len([f for f in os.listdir(split_pred_nifti_path) if f.endswith('.nii.gz')])
  df_list = []
  for n in range(0,num_labels):
    mask_path = os.path.join(split_pred_nifti_path, label_names[n] + '.nii.gz')
    # Run the extractor
    print(f"ct nifti path : {ct_nifti_path}")
    print(f" mask path : {mask_path}")
    result = extractor.execute(ct_nifti_path, mask_path) # dictionary
    # keep only the features we want
    # Get the corresponding label number -- all might not be present
    corresponding_label_value = label_values[label_names.index(label_names[n])]
    dict_keep = {'ReferencedSegment': corresponding_label_value,
                 'label_name': label_names[n]}
    keys_keep = [f for f in result.keys() if 'original_shape' in f]
    # Just keep the feature keys we want
    dict_keep_new_values = {key_keep: result[key_keep] for key_keep in keys_keep}
    dict_keep.update(dict_keep_new_values)
    df1 = pd.DataFrame([dict_keep])
    # change values of columns to remove original_shape_
    df1.columns = df1.columns.str.replace('original_shape_', '')
    # Append to the ReferencedSegment and label_name df
    df_list.append(df1)

  # concat all label features
  df = pd.concat(df_list)

  return df

In [ ]:
def order_dicom_files_image_position(dcm_directory):
  """
  Orders the dicom files according to image position and orientation.

  Arguments:
    dcm_directory : input directory of dcm files to put in order

  Outputs:
    files_sorted   : dcm files in sorted order
    sop_all_sorted : the SOPInstanceUIDs in sorted order
    pos_all_sorted : the image position in sorted order

  """
  files = [os.path.join(dcm_directory,f) for f in os.listdir(dcm_directory)]

  num_files = len(files)

  pos_all = []
  sop_all = []

  for n in range(0,num_files):
    # read dcm file
    filename = files[n]
    ds = dcmread(filename)

    # get ImageOrientation (0020, 0037)
    # ImageOrientation = ds['0x0020','0x0037'].value
    ImageOrientation = ds.ImageOrientationPatient

    # get ImagePositionPatient (0020, 0032)
    # ImagePositionPatient = ds['0x0020','0x0032'].value
    ImagePositionPatient = ds.ImagePositionPatient

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID
    # sop = ds['0x0008', '0x0018'].value
    sop = ds.SOPInstanceUID
    sop_all.append(sop)


  #----- order the SOPInstanceUID/files by z value ----#

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  return files_sorted, sop_all_sorted, pos_all_sorted

In [ ]:
def create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                         SOPInstanceUID_seg,
                                                         seg_file,
                                                         dcm_directory,
                                                         segments_code_mapping_df,
                                                         shape_features_code_mapping_df,
                                                         df_features,
                                                         SegmentAlgorithmName
                                                         ):

  """Function that creates the metajson necessary for the creation of a
  structured report from a pandas dataframe of label names and features for
  each.

  Inputs:
    SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                      file
    SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
    seg_file                        : filename of SEG DCM file
    dcm_directory                   : ct directory that will be sorted in
                                      terms of axial ordering according to the
                                      ImagePositionPatient and ImageOrientation
                                      fields
    segments_code_mapping_df        : dataframe that holds the names of the
                                      segments and the associated code values etc.
    shape_features_code_mapping_df  : dataframe that holds the names of the
                                      features and the associated code values etc.
    df_features                     : a pandas dataframe holding the segments and a
                                      set of 3D shape features for each
    SegmentAlgorithmName            : the name of the algorithm used to create the
                                      segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report
  """

  # --- Get the version number for the pyradiomics package --- #

  pyradiomics_version_number = str(radiomics.__version__)

  # --- Sort the dcm files first according to --- #
  # --- ImagePositionPatient and ImageOrientation --- #

  files_sorted, sop_all_sorted, pos_all_sorted = order_dicom_files_image_position(dcm_directory)
  files_sorted = [os.path.basename(f) for f in files_sorted]

  # --- Create the header for the json --- #

  inputMetadata = {}
  inputMetadata["@schema"]= "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/sr-tid1500-schema.json#"
  # inputMetadata["SeriesDescription"] = "Measurements"
  inputMetadata["SeriesDescription"] = SegmentAlgorithmName + '_' + "Measurements"
  inputMetadata["SeriesNumber"] = "1001"
  inputMetadata["InstanceNumber"] = "1"

  inputMetadata["compositeContext"] = [seg_file] # not full path

  inputMetadata["imageLibrary"] = files_sorted # not full path

  # inputMetadata["observerContext"] = {
  #                                     "ObserverType": "PERSON",
  #                                     "PersonObserverName": "Reader1"
  #                                   }
  # inputMetadata["observerContext"] = {
  #                     "ObserverType": "DEVICE",
  #                     "DeviceObserverName": "pyradiomics",
  #                     "DeviceObserverModelName": "v3.0.1"
  #                   }
  inputMetadata["observerContext"] = {
                      "ObserverType": "DEVICE",
                      "DeviceObserverName": "pyradiomics",
                      "DeviceObserverModelName": pyradiomics_version_number
                    }

  inputMetadata["VerificationFlag"]  = "UNVERIFIED"
  inputMetadata["CompletionFlag"] =  "COMPLETE"
  inputMetadata["activitySession"] = "1"
  inputMetadata["timePoint"] = "1"

  # ------------------------------------------------------------------------- #
  # --- Create the measurement_dict for each segment - holds all features --- #

  measurement = []

  # --- Now create the dict for all features and all segments --- #

  # --- Loop over the number of segments --- #

  # number of rows in the df_features
  num_segments = df_features.shape[0]

  # Array of dictionaries - one dictionary for each segment
  measurement_across_segments_combined = []

  for segment_id in range(0,num_segments):

    ReferencedSegment = df_features['ReferencedSegment'].values[segment_id]
    FindingSite = df_features['label_name'].values[segment_id]

    print('segment_id: ' + str(segment_id))
    print('ReferencedSegment: ' + str(ReferencedSegment))
    print('FindingSite: ' + str(FindingSite))

    # --- Create the dict for the Measurements group --- #
    TrackingIdentifier = "Measurements group " + str(ReferencedSegment)

    segment_row = segments_code_mapping_df[segments_code_mapping_df["segment"] == FindingSite]
    # print(segment_row)

    my_dict = {
      "TrackingIdentifier": str(TrackingIdentifier),
      "ReferencedSegment": int(ReferencedSegment),
      "SourceSeriesForImageSegmentation": str(SeriesInstanceUID),
      "segmentationSOPInstanceUID": str(SOPInstanceUID_seg),
      "Finding": {
        "CodeValue": "113343008",
        "CodingSchemeDesignator": "SCT",
        "CodeMeaning": "Organ"
      },
      "FindingSite": {
        "CodeValue": str(segment_row["FindingSite_CodeValue"].values[0]),
        "CodingSchemeDesignator": str(segment_row["FindingSite_CodingSchemeDesignator"].values[0]),
        "CodeMeaning": str(segment_row["FindingSite_CodeMeaning"].values[0])
      }
    }

    measurement = []
    # number of features - number of columns in df_features - 2 (label_name and ReferencedSegment)
    num_values = len(df_features.columns)-2

    feature_list = df_features.columns[2:] # remove first two


    # For each measurement per region segment
    for n in range(0,num_values):
      measurement_dict = {}
      row = df_features.loc[df_features['label_name'] == FindingSite]
      feature_row = shape_features_code_mapping_df.loc[shape_features_code_mapping_df["shape_feature"] == feature_list[n]]
      value = str(np.round(row[feature_list[n]].values[0],3))
      measurement_dict["value"] = value
      measurement_dict["quantity"] = {}
      measurement_dict["quantity"]["CodeValue"] = str(feature_row["quantity_CodeValue"].values[0])
      measurement_dict["quantity"]["CodingSchemeDesignator"] = str(feature_row["quantity_CodingSchemeDesignator"].values[0])
      measurement_dict["quantity"]["CodeMeaning"] = str(feature_row["quantity_CodeMeaning"].values[0])
      measurement_dict["units"] = {}
      measurement_dict["units"]["CodeValue"] = str(feature_row["units_CodeValue"].values[0])
      measurement_dict["units"]["CodingSchemeDesignator"] = str(feature_row["units_CodingSchemeDesignator"].values[0])
      measurement_dict["units"]["CodeMeaning"] = str(feature_row["units_CodeMeaning"].values[0])
      measurement_dict["measurementAlgorithmIdentification"] = {}
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmName"] = "pyradiomics"
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmVersion"] = str(pyradiomics_version_number)
      measurement.append(measurement_dict)

    measurement_combined_dict = {}
    measurement_combined_dict['measurementItems'] = measurement # measurement is an array of dictionaries

    output_dict_one_segment = {**my_dict, **measurement_combined_dict}

    # append to array for all segments

    measurement_across_segments_combined.append(output_dict_one_segment)

  # --- Add the measurement data --- #

  inputMetadata["Measurements"] = {}
  inputMetadata["Measurements"] = measurement_across_segments_combined

  return inputMetadata

In [ ]:
def save_structured_report_for_shape_features(SeriesInstanceUID,
                                              SOPInstanceUID_seg,
                                              pred_dicomseg_path,
                                              dicomseg_json_path,
                                              dcm_directory,
                                              pred_nifti_path,
                                              nnunet_base_path,
                                              ct_nifti_path,
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path,
                                              SegmentAlgorithmName
                                              ):

  """ This function creates the SR necessary for the nnUNet shape features

  Inputs:
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                    file
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
  pred_dicomseg_path              : filename of DICOM SEG file
  dicomseg_json_path              : json file for DICOM SEG file
  dcm_directory                   : list of ct files that will be sorted in
                                    terms of axial ordering according to the
                                    ImagePositionPatient and ImageOrientation
                                    fields
  pred_nifti_path                 : predictions in nifti format
  nnunet_base_path                : path to hold the split nifti files
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the
                                    features and the associated code values etc.
  sr_json_path                    : the path that the metajson for the SR for
                                    the 3D shape features will be saved
  sr_path                         : the path that the SR for the 3D shape
                                    features will be saved
  SegmentAlgorithmName            : the name of the algorithm used to create the
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = get_label_and_names_from_metadata_json(dicomseg_json_path)

  # # --- split the multilabel nifti into individual files --- #
  split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  if not os.path.isdir(split_pred_nii_path):
    os.mkdir(split_pred_nii_path)
  split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # --- compute features and save csv for each region --- #
  # if not os.path.isdir(os.environ["nnUNet_preds_features"]):
  #   os.mkdir(os.environ["nnUNet_preds_features"])
  df_features = compute_pyradiomics_3D_features(ct_nifti_path,
                                                label_values,
                                                label_names,
                                                split_pred_nii_path,
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')

  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  # !rm $split_pred_nii_path/*
  # remove csv
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path,
                                                                       dcm_directory,
                                                                       segments_code_mapping_df,
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features,
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- #
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json))
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputCompositeContextDirectory $inputCompositeContextDirectory \
                --inputMetadata $inputMetadata_json
  print ('wrote out SR for shape features')

  return

In [ ]:
def save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID,
                                              label_values,
                                              label_names,
                                              SOPInstanceUID_seg,
                                              pred_dicomseg_path,
                                              dicomseg_json_path,
                                              dcm_directory,
                                              pred_nifti_path,
                                              split_pred_nii_path,
                                              ct_nifti_path,
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path,
                                              SegmentAlgorithmName
                                              ):

  """ This function creates the SR necessary for the nnUNet shape features

  Inputs:
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                    file
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
  pred_dicomseg_path              : filename of DICOM SEG file
  dicomseg_json_path              : json file for DICOM SEG file
  dcm_directory                   : list of ct files that will be sorted in
                                    terms of axial ordering according to the
                                    ImagePositionPatient and ImageOrientation
                                    fields
  pred_nifti_path                 : predictions in nifti format
  nnunet_base_path                : path to hold the split nifti files
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the
                                    features and the associated code values etc.
  sr_json_path                    : the path that the metajson for the SR for
                                    the 3D shape features will be saved
  sr_path                         : the path that the SR for the 3D shape
                                    features will be saved
  SegmentAlgorithmName            : the name of the algorithm used to create the
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = label_values, label_names#get_label_and_names_from_metadata_json(dicomseg_json_path)

  # # --- split the multilabel nifti into individual files --- #
  # split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  # if not os.path.isdir(split_pred_nii_path):
  #   os.mkdir(split_pred_nii_path)
  # split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # # --- compute features and save csv for each region --- #
  # if not os.path.isdir(os.environ["nnUNet_preds_features"]):
  #   os.mkdir(os.environ["nnUNet_preds_features"])
  df_features = compute_pyradiomics_3D_features(ct_nifti_path,
                                                label_values,
                                                label_names,
                                                split_pred_nii_path,
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')

  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  !rm $split_pred_nii_path/*
  # remove csv
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path,
                                                                       dcm_directory,
                                                                       segments_code_mapping_df,
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features,
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- #
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json))
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputCompositeContextDirectory $inputCompositeContextDirectory \
                --inputMetadata $inputMetadata_json
  print ('wrote out SR for shape features')

  return

In [ ]:
def save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID,
                                              label_values,
                                              label_names,
                                              SOPInstanceUID_seg,
                                              pred_dicomseg_path,
                                              dicomseg_json_path,
                                              dcm_directory,
                                              pred_nifti_path,
                                              split_pred_nii_path,
                                              ct_nifti_path,
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path,
                                              SegmentAlgorithmName
                                              ):

  """ This function creates the SR necessary for the nnUNet shape features

  Inputs:
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                    file
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
  pred_dicomseg_path              : filename of DICOM SEG file
  dicomseg_json_path              : json file for DICOM SEG file
  dcm_directory                   : list of ct files that will be sorted in
                                    terms of axial ordering according to the
                                    ImagePositionPatient and ImageOrientation
                                    fields
  pred_nifti_path                 : predictions in nifti format
  nnunet_base_path                : path to hold the split nifti files
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the
                                    features and the associated code values etc.
  sr_json_path                    : the path that the metajson for the SR for
                                    the 3D shape features will be saved
  sr_path                         : the path that the SR for the 3D shape
                                    features will be saved
  SegmentAlgorithmName            : the name of the algorithm used to create the
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = label_values, label_names#get_label_and_names_from_metadata_json(dicomseg_json_path)

  # # --- split the multilabel nifti into individual files --- #
  # split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  # if not os.path.isdir(split_pred_nii_path):
  #   os.mkdir(split_pred_nii_path)
  # split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # # --- compute features and save csv for each region --- #
  # if not os.path.isdir(os.environ["nnUNet_preds_features"]):
  #   os.mkdir(os.environ["nnUNet_preds_features"])
  df_features = compute_pyradiomics_3D_features(ct_nifti_path,
                                                label_values,
                                                label_names,
                                                split_pred_nii_path,
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')

  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  !rm $split_pred_nii_path/*
  # remove csv
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path,
                                                                       dcm_directory,
                                                                       segments_code_mapping_df,
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features,
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- #
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json))
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputMetadata $inputMetadata_json
                # --inputCompositeContextDirectory $inputCompositeContextDirectory \ ##fails for now
  print ('wrote out SR for shape features')

  return

# Data selection, downloading and structuring -- Conversion to DICOM

We want to select here the collection named qin-prostate repeatibility, and more particularly the two timepoints per patient ID for further analysis.

In order to use data hosted by IDC effectively, you will need to utilize metadata to navigate what data is available and to select specific files that are relevant in your analysis. The main metadata table you will need for this purpose is the [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table) table.

This query has one row per file hosted by IDC. All of IDC data is in DICOM format, and each of the rows in this table will have all of the DICOM attributes extracted from a given file. It will also have various columns containing non-DICOM metadata, such as the name of the collection where the file is included, size of the file, and URL that can be used to retrieve that file.

To query IDC BigQuery tables, you can use one of the following approaches:
1. `%%bigquery` magic will allow you to define your query in plain SQL, and load the result of the query into a Pandas dataframe.
2. [BigQuery Python API](https://googleapis.dev/python/bigquery/latest/index.html) is more flexible in allowing you to parameterize your query.
3. [Google Cloud BigQuery console](https://console.cloud.google.com/bigquery) is very convenient for interactive query exploration of tables.
4. [`gcloud bq`](https://cloud.google.com/bigquery/docs/bq-command-line-tool) is the command line tool that comes as part of [Cloud SDK](https://cloud.google.com/sdk) and is convenient for scripting interactions from the shell. Cloud SDK is preinstalled on Colab.

In the following cells we will utilize `%%bigquery`, Python BigQuery SDK and BigQuery console for working with IDC BigQuery tables.

First, to verify that you are authenticated, and your project ID is working, let's run a test query against IDC BigQuery table to get the summary statistics about the  of data available in IDC.


Given `SeriesInstanceUID` value identifying the image series, we can query the IDC metadata table to get the list of files (defined by the Google Storage URLs) corresponding to this series.

All of the DICOM metadata for each of the DICOM files is available in the BigQuery table we will be querying. We will get not just the `gcs_url`, but also identifiers for the Study, Series and Instance, to better understand organization of data, and since `StudyInstanceUID` will be handy later when we get to the visualization of the data.

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])

In [ ]:
from google.cloud import bigquery
selection_query = f"""
WITH
  prostatex_seg AS (
  SELECT
    DISTINCT(ReferencedSeriesSequence[SAFE_OFFSET(0)].SeriesInstanceUID) AS RefSerieUID
  FROM
    `bigquery-public-data.idc_v15.dicom_all`
  WHERE
    collection_id = 'prostatex'
    AND SegmentSequence[SAFE_OFFSET(0)].SegmentedPropertyTypeCodeSequence[SAFE_OFFSET(0)].CodeMeaning = 'Peripheral zone of prostate')
SELECT
  dc_all.*
FROM
  `bigquery-public-data.idc_v15.dicom_all` AS dc_all
JOIN
  prostatex_seg
ON
  dc_all.SeriesInstanceUID = prostatex_seg.RefSerieUID
WHERE
  dc_all.collection_id = 'prostatex'
  AND dc_all.Modality = 'MR'
  AND LOWER(dc_all.SeriesDescription) LIKE '%t2%'
ORDER BY PatientID"""
selection_result = bq_client.query(selection_query)
selection_t2_df = selection_result.result().to_dataframe()

In [ ]:
from google.cloud import bigquery
selection_query = f"""
SELECT
    *,
    ReferencedSeriesSequence[SAFE_OFFSET(0)].SeriesInstanceUID as RefSerieUID
  FROM
    `bigquery-public-data.idc_v14.dicom_all`
  WHERE
    SegmentSequence[SAFE_OFFSET(0)].SegmentedPropertyTypeCodeSequence[SAFE_OFFSET(0)].CodeMeaning = 'Peripheral zone of prostate'
    AND Modality='SEG'
    """
selection_result = bq_client.query(selection_query)
selection_t2_seg_prostate_df = selection_result.result().to_dataframe()

In [ ]:
len(selection_t2_df.SeriesInstanceUID.unique())

98

In [ ]:
len(selection_t2_seg_prostate_df.SeriesInstanceUID.unique())

98

Get SerieUIDs in GCP bucket processed by nnunet AI model -- structured reports  -- RefSeieUID.dcm

In [ ]:
!rm bucketUIDs_processed.csv
!gcloud storage ls --recursive gs://prostate_eval_idc_neurips_2024/dicom_seg_ai_results/prostate158_prostatex_zonal_dcm_seg/* > bucketUIDs_processed.csv
sereUID_processed_dcm = pd.read_csv("bucketUIDs_processed.csv", names=["RefSerieUID"], skiprows=[0])
seriesInstanceUID_dcm_processed_lst = [x.split("/")[-1].split("_")[1].replace(".dcm", "") for x in sereUID_processed_dcm.RefSerieUID.values]

In [ ]:
sereUID_processed_dcm

,RefSerieUID


In [ ]:
serieUID_current_lst = list(set(selection_t2_df.SeriesInstanceUID.values) - set(seriesInstanceUID_dcm_processed_lst))

In [ ]:
len(serieUID_current_lst)

98

# Download json metadata template for conversion of nnunet preds into DICOM

In [ ]:
seg_whole_prostate_json = "https://www.dropbox.com/scl/fi/hokxewmdbtgkkidce43t4/pred_metadata_prostate158_all.json?rlkey=4mk95vty1oz61eo07mwlj4cfq&dl=0"
seg_dcm_metadata_json_file = "pred_metadata.json"#"/content/nnUnet/models/Task024_Promise.zip"
!wget -O $seg_dcm_metadata_json_file $seg_whole_prostate_json

--2024-06-10 02:24:55--  https://www.dropbox.com/scl/fi/hokxewmdbtgkkidce43t4/pred_metadata_prostate158_all.json?rlkey=4mk95vty1oz61eo07mwlj4cfq
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb24ba9486b191af84345456f5b.dl.dropboxusercontent.com/cd/0/inline/CUgpqsMSmkD0sEzLmSgND0mc7TDcY8VdV2lEJfTfPUMoWPGxcaThG9mfnklTbjDDUjKUaRyaaFbB6R-F_aDLDr_wiPcyF3vK7j9JDcY7c2TmiRQzo4qH1QVeaCQRDyY7hlQ3HsUHzvTncoiOoZcvqGhr/file# [following]
--2024-06-10 02:24:56--  https://ucb24ba9486b191af84345456f5b.dl.dropboxusercontent.com/cd/0/inline/CUgpqsMSmkD0sEzLmSgND0mc7TDcY8VdV2lEJfTfPUMoWPGxcaThG9mfnklTbjDDUjKUaRyaaFbB6R-F_aDLDr_wiPcyF3vK7j9JDcY7c2TmiRQzo4qH1QVeaCQRDyY7hlQ3HsUHzvTncoiOoZcvqGhr/file
Resolving ucb24ba9486b191af84345456f5b.dl.dropboxusercontent.com (ucb24ba9486b191af84345456f5b.dl.dropboxusercontent.com)..

# Download and install MONAI and pre-trained model

nnUnet pre-trained models zip files can also be found [here](https://zenodo.org/record/4003545#.YsWmH-zMLt8).

In [ ]:
!wget https://zenodo.org/record/7040585/files/anatomy.pt?download=1 -O {os.environ['monai_workspace_model_path']}
# !zenodo_get 7040585
# !mv ./anatomy.pt {os.environ["monai_workspace_model_dir"]}/anatomy.pt

--2024-06-10 02:24:58--  https://zenodo.org/record/7040585/files/anatomy.pt?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7040585/files/anatomy.pt [following]
--2024-06-10 02:24:59--  https://zenodo.org/records/7040585/files/anatomy.pt
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 152810901 (146M) [application/octet-stream]
Saving to: ‘monai_workspace/models/anatomy.pt’

monai_workspace/mod 100%[===================>] 145.73M  22.1MB/s    in 8.0s    

2024-06-10 02:25:07 (18.3 MB/s) - ‘monai_workspace/models/anatomy.pt’ saved [152810901/152810901]



We process each SerieUID one at a time, we first retrieve serieUIDs processed in GCP bucket and subtract that amount from the serieUIDs retrieved from IDC MR T2 series, obtaining the non processed serieUID_current_lst.

If that is not the desired behaviour, change serieUID_current_lst below to list((selection_df.SeriesInstanceUID.unique())

# Main loop

In [ ]:
def reset_folders():
  for key, path in os.environ.items():
    check_patterns = [True for el in ["prostatex", "IDC_Downloads", "IDC_Downloads_Sorted", \
                                      "prostatex_dicom", "prostatex_root",
                                       "stl_labelmap", "mri_us_biopsy_analysis", "IDC", "monai_workspace_eval", "monai_workspace_dataset"] if el in key]
    if True in check_patterns:
      !rm -rf $path
      !mkdir -p $path

In [ ]:
#add to BQ table
client = bigquery.Client()
table_id = "idc-sandbox-003.prostate_eval_neurips_2024.prostate158_prostatex_zonal_eval_results_table"
#whole process
for serieUID_current in serieUID_current_lst:#serieUID_current_lst:#list((selection_df.SeriesInstanceUID.unique())
  #reset processing folders
  reset_folders()
  idc_df = download_idc_data_serie_uid(idc_t2_df=selection_t2_df[selection_t2_df.SeriesInstanceUID \
                                                          == serieUID_current], idc_downloads_path=os.environ["IDC_IMG_Downloads"],
                                      idc_downloads_sorted_path=os.environ["IDC_IMG_Downloads_Sorted"],
                                      idc_dicom_path=os.environ["prostatex_dicom"],
                                      idc_nii_path=os.environ["prostatex_nii"])
  ##reformat idc input data
  patID = idc_df[idc_df["SeriesInstanceUID"] == serieUID_current].PatientID.unique()[0]
  reformat_image_nnunet(patID, serieUID_current, images_nii_folder = os.environ["prostatex_nii"], out_path_folder=os.environ["prostatex_nii_formatted"])
  #setup monai workspace
  test_df = pd.DataFrame(columns=["t2", "t2_label"])#t2 is serieUID of reference t2 image
  !cp -r {os.environ['prostatex_nii_formatted']}/*.nii.gz {os.environ["monai_workspace_dataset"]}/
  test_df.to_csv(os.path.join(os.environ["monai_workspace_dataset"], "test.csv"))
  inference_json = json.load(open("/content/model-zoo/models/prostate_mri_anatomy/configs/inference.json"))#path to downloaded bundle from git monai bundles prostate158
  #setup variables
  inference_json["bundle_root"] = os.environ["monai_workspace"]
  inference_json["output_dir"] = os.environ["monai_workspace_eval"]
  inference_json["dataset_dir"] = os.environ["monai_workspace_dataset"]
  inference_json["handlers"][0]["load_path"] = os.environ["monai_workspace_model_path"]
  datalist_json = glob.glob(os.path.join(os.environ["monai_workspace_dataset"], "*.nii.gz"))
  inference_json["datalist"] = datalist_json
  with open(f"{os.environ['monai_workspace']}/inference.json", 'w') as fp:
      json.dump(inference_json, fp)
  #inference
  config = ConfigParser()
  config.read_config(f"{os.environ['monai_workspace']}/inference.json")
  now = time.time()
  !python -m monai.bundle run evaluating --meta_file /content/model-zoo/models/prostate_mri_anatomy/configs/metadata.json --config_file {os.environ['monai_workspace']}/inference.json
  time_pred = int(time.time() - now)
  # # #resample preds data
  resample_preds(input_path_nnunet_preds=os.environ["monai_workspace_eval"],\
                  input_path_t2_idc=os.environ["prostatex_nii"], output_path=os.environ["monai_workspace_eval_resampled"], suffix_ref=".nii.gz")
  #split preds data for analysis -- in case of multiple segments
  split_seg_nii(input_labelmap=glob.glob(f"{os.environ['monai_workspace_eval_resampled']}/*.nii.gz")[0],
                output_folder_path= os.environ["monai_workspace_eval_resampled_split"],dic_labelID={"Transition zone of prostate" : 1, "Peripheral zone of prostate" : 2})
  #combine pz and central gland to form prostate
  combine_prostate_regions(input_path_nii_pred=glob.glob(f"{os.environ['monai_workspace_eval_resampled']}/*.nii.gz")[0],
  output_path_folder=os.environ["monai_workspace_eval_resampled_split"], filename="Prostate.nii.gz")
  # #convert nnunet_preds to dicom
  patID = selection_t2_df[selection_t2_df["SeriesInstanceUID"] == serieUID_current].PatientID.unique()[0]
  studyUID = selection_t2_df[selection_t2_df["SeriesInstanceUID"] == serieUID_current].StudyInstanceUID.unique()[0]
  #Conversion to dcm
  ## reformat labelID to 1,23 in monai_split_dir
  change_labelID_seg_nii(glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Transition*.nii.gz")[0], 1, os.environ['monai_workspace_eval_resampled_split_labelID'])
  change_labelID_seg_nii(glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Peripheral*.nii.gz")[0], 2, os.environ['monai_workspace_eval_resampled_split_labelID'])
  change_labelID_seg_nii(glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Prostate.nii.gz")[0], 3, os.environ['monai_workspace_eval_resampled_split_labelID'])
  list_seg_labelID = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/*.nii.gz")
  t2_dcm_folder = glob.glob(f"{os.environ['prostatex_dicom']}/{patID}/{studyUID}/{serieUID_current}")[0]
  output_path = f"{os.environ['monai_workspace_eval_resampled_dcm']}/{patID}_{serieUID_current}.dcm"
  prostate_nii = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/Prostate.nii.gz")[0]
  tz_nii = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/Transition*.nii.gz")[0]
  pz_nii = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/Peripheral*.nii.gz")[0]
  !itkimage2segimage --inputImageList '{tz_nii}','{pz_nii}','{prostate_nii}' \
    --inputDICOMDirectory  $t2_dcm_folder \
    --inputMetadata  $seg_dcm_metadata_json_file \
    --outputDICOM $output_path
  #convert dicom nnunet preds back to nifti
  convert_seg_to_nii(input_path=glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0], \
                    output_path=os.environ['monai_workspace_eval_resampled_dcm_to_nifti'])
  idc_seg_data_df = selection_t2_seg_prostate_df[selection_t2_seg_prostate_df.RefSerieUID \
                                                          == serieUID_current]
  download_idc_data_serie_uid_seg(df_in=idc_seg_data_df,
                                      out_path=os.environ["prostatex_dicom_seg"],
                                      idc_downloads_path=os.environ["IDC_Downloads"],
                                  idc_downloads_sorted_path=os.environ["IDC_Downloads_Sorted"])
  ##convert seg dcm to nii
  convert_seg_to_nii(input_path=glob.glob(f"{os.environ['prostatex_dicom_seg']}/**/*.dcm", recursive=True)[0], \
                    output_path=os.environ['prostatex_nii_seg'])
  # # reformat seg to codeMeaning.nii.gz
  for x in json.load(open(glob.glob(os.path.join(os.environ["prostatex_nii_seg"], "meta.json"))[0]))["segmentAttributes"]:
    labelid = x[0]["labelID"]
    SegmentedPropertyTypeCodeSequenceCodeMeaning = x[0]["SegmentedPropertyTypeCodeSequence"]["CodeMeaning"]
    # Starting with SimpleITK
    img = sitk.ReadImage(f"{os.environ['prostatex_nii_seg']}/{int(labelid)}.nii.gz")
    # Custom Python code working on a numpy array.
    npa_res = sitk.GetArrayFromImage(img)
    npa_res[npa_res != 0] = 1 #convert labelID to 1
    # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
    res_img = sitk.GetImageFromArray(npa_res)
    res_img.CopyInformation(img)
    sitk.WriteImage(res_img, f"{os.environ['prostatex_nii_seg_formatted']}/{SegmentedPropertyTypeCodeSequenceCodeMeaning}.nii.gz")
  t2_nii_path = glob.glob(os.path.join(os.environ["prostatex_nii"], f"{patID}_{serieUID_current}*.nii.gz"))[0]
  resample_idc_seg_nii(os.environ['prostatex_nii_seg_formatted'], os.environ["prostatex_nii_seg_resampled"], t2_nii_path)
  ###################################
  seg_gold_serieUID = str(pydicom.dcmread(glob.glob(f"{os.environ['prostatex_dicom_seg']}/**/*.dcm", recursive=True)[0]).SeriesInstanceUID)
  pred_nnunet_serieUID = pydicom.dcmread(glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0]).SeriesInstanceUID
  seg_pred_nnunet_sopInstanceUID = str(pydicom.dcmread(glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0]).SOPInstanceUID)
  dicomseg_json_path_modified = glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm_to_nifti']}/*.json")[0]
  for stl_seg_nii in glob.glob(f"{os.environ['prostatex_nii_seg_resampled']}/*.nii.gz"):
    assert sorted(np.unique(sitk.GetArrayFromImage(sitk.ReadImage(stl_seg_nii))))[1] == 1
    labelMeaning = stl_seg_nii.split('/')[-1].replace(".nii.gz", "")
    ai_nii_path = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/{labelMeaning}*.nii.gz")
    if len(ai_nii_path) >=1:
      ai_nii_path = ai_nii_path[0]
      rows_to_insert = [
        {"refSerieUID": serieUID_current,
        "ai_serieUID" : pred_nnunet_serieUID,
        "gold_serieUID" : seg_gold_serieUID,
        "segment" : labelMeaning,
        "dsc": compute_dice(ai_nii_path, stl_seg_nii),
        "hsdff" : compute_hsdff_regular(ai_nii_path, stl_seg_nii),
        "hsdff_95" : compute_hsdff_95(ai_nii_path, stl_seg_nii),
        "tp" : compute_true_positive(ai_nii_path, stl_seg_nii),
        "fp" : compute_false_positive(ai_nii_path, stl_seg_nii),
        "tn" : compute_true_negative(ai_nii_path, stl_seg_nii),
        "fn" : compute_false_negative(ai_nii_path, stl_seg_nii),
        "se" : compute_sensitivity(ai_nii_path, stl_seg_nii),
        "sp" : compute_specificity(ai_nii_path, stl_seg_nii),
        "prediction_time" : float(time_pred)
        }
        ]
      try:
        asd  = round(float(compute_avg_surface_dist(ai_nii_path, stl_seg_nii)[0]['msd'][0]),2)
      except:
        asd = -999.
      rows_to_insert[0]["asd"] = asd
      print(rows_to_insert)
      errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
      if errors == []:
          print("New rows have been added.")
      else:
          print("Encountered errors while inserting rows: {}".format(errors))
  #SR DICOM
  # #get dcm dir
  ct_nifti_path = glob.glob(f"{os.environ['prostatex_nii']}/{patID}_{serieUID_current}*.nii.gz")[0]
  pred_path = glob.glob(f"{os.environ['monai_workspace_eval_resampled']}/*.nii.gz")[0]
  try:
    nnunet_features_metajson = save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID=serieUID_current,
                                                                      SOPInstanceUID_seg=seg_pred_nnunet_sopInstanceUID,
                                                                      label_values=[1,2,3],
                                                                      label_names = ["Transition zone of prostate", "Peripheral zone of prostate", "Prostate"],
                                                                      pred_dicomseg_path=glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0],
                                                                      dicomseg_json_path=dicomseg_json_path_modified,
                                                                      dcm_directory=t2_dcm_folder,
                                                                      pred_nifti_path=pred_path,
                                                                      split_pred_nii_path=os.environ["monai_workspace_eval_resampled_split"],
                                                                      ct_nifti_path=ct_nifti_path,
                                                                      segments_code_mapping_df=idc_seg_segments_code_mapping_df,
                                                                      shape_features_code_mapping_df=nnunet_shape_features_code_mapping_df,
                                                                      sr_json_path=os.path.join(os.environ['monai_workspace_eval_sr_json'], serieUID_current + '_SR.json'),
                                                                      sr_path=os.path.join(os.environ['monai_workspace_eval_sr'], serieUID_current + '_SR.dcm'),
                                                                      SegmentAlgorithmName='MONAI-prostate158-zonal-prostatex-measurements')
  except:
    print("SR COMPUTATION DID NOT GO THROUGH")
  # ###### UPLOAD RESULTS
  out_nnunet_dcm_seg = os.environ['monai_workspace_eval_resampled_dcm']
  out_nnunet_sr_seg = os.environ['monai_workspace_eval_sr']
  # out_stl_gold_sr_seg = os.environ['stl_labelmaps_sr']
  !gsutil cp -r {out_nnunet_dcm_seg+'/*.dcm'} gs://prostate_eval_idc_neurips_2024/dicom_seg_ai_results/prostate158_prostatex_zonal_dcm_seg/
  !gsutil cp -r {out_nnunet_sr_seg+'/*.dcm'} gs://prostate_eval_idc_neurips_2024/dicom_seg_ai_results/prostate158_prostatex_zonal_dcm_sr/
  # except:
  #   print("\n")
  #   print("------------------------------------------------------------------------")
  #   print(f"DID NOT WORK for patientID {patID} and serieUID : {serieUID_current}")
  #   print("\n")

Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/8a1d2fd5-f4ea-439b-b518-57a4aeed00c4.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/6b2e5daf-3861-441a-ad0a-38a988a92541.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/9e6e0e70-5ce2-4564-ba16-7fad54f394db.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/04c8259c-4434-4503-9fc3-0fd5c5c25c8a.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/395981dd-21d9-40bb-b412-f7d052eb309f.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/1d507893-e23c-4f70-918b-5c0765d5213f.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/d7fb1390-1c5a-463a-b5a3-3a8e19543af3.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/92f4079b-f2bc-420a-b52b-30b2f49ed545.dcm...
Copying gs://public-datasets-idc/ec410719-9562-4075-9817-bbd0e04206d5/6c07928f-f7ed-48a7

INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'correctMask': True, 'geometryTolerance': 0.01}
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


[{'refSerieUID': '1.3.6.1.4.1.14519.5.2.1.7311.5101.186553403053805718209125341361', 'ai_serieUID': '1.2.276.0.7230010.3.1.3.481037312.6495.1717986363.947105', 'gold_serieUID': '1.2.276.0.7230010.3.1.3.1070885483.15088.1599120352.564', 'segment': 'Transition zone of prostate', 'dsc': 0.936611, 'hsdff': 7.483315, 'hsdff_95': 0.5, 'tp': '117082', 'fp': '11146', 'tn': '2979795', 'fn': '4702', 'se': '0.961391', 'sp': '0.996273', 'prediction_time': 38.0, 'asd': 0.63}]
New rows have been added.
ct nifti path : /content/prostateX/nii/ProstateX-0070_1.3.6.1.4.1.14519.5.2.1.7311.5101.186553403053805718209125341361.nii.gz
 mask path : monai_workspace/eval_resampled_split/Transition zone of prostate.nii.gz


INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


ct nifti path : /content/prostateX/nii/ProstateX-0070_1.3.6.1.4.1.14519.5.2.1.7311.5101.186553403053805718209125341361.nii.gz
 mask path : monai_workspace/eval_resampled_split/Peripheral zone of prostate.nii.gz


...

ct nifti path : /content/prostateX/nii/ProstateX-0066_1.3.6.1.4.1.14519.5.2.1.7311.5101.252385246988168654915352850239.nii.gz
 mask path : monai_workspace/eval_resampled_split/Peripheral zone of prostate.nii.gz


INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


ct nifti path : /content/prostateX/nii/ProstateX-0066_1.3.6.1.4.1.14519.5.2.1.7311.5101.252385246988168654915352850239.nii.gz
 mask path : monai_workspace/eval_resampled_split/Prostate.nii.gz


INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image


created df_features
segment_id: 0
ReferencedSegment: 1
FindingSite: Transition zone of prostate
segment_id: 1
ReferencedSegment: 2
FindingSite: Peripheral zone of prostate
segment_id: 2
ReferencedSegment: 3
FindingSite: Prostate
created SR json for shape features
wrote out json for shape features
inputImageLibraryDirectory: /content/prostateX/dicom/ProstateX-0066/1.3.6.1.4.1.14519.5.2.1.7311.5101.320379732448304997639520720109/1.3.6.1.4.1.14519.5.2.1.7311.5101.252385246988168654915352850239
outputDICOM: monai_workspace/structured_reports/1.3.6.1.4.1.14519.5.2.1.7311.5101.252385246988168654915352850239_SR.dcm
inputCompositeContextDirectory: ProstateX-0066_1.3.6.1.4.1.14519.5.2.1.7311.5101.252385246988168654915352850239.dcm
inputMetadata_json: monai_workspace/structured_reports_json/1.3.6.1.4.1.14519.5.2.1.7311.5101.252385246988168654915352850239_SR.json
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Total measurement groups: 3
Adding to compositeConte